In [4]:
# Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os  # Import the os module

In [5]:
# Set data directory to the path where you saved your dataset
data_dir = r'D:\Food Image processing\food-101\images'  # Use raw string or double backslashes

# Verify the directory contents
print(os.listdir(data_dir))

['.DS_Store', 'apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macaron

In [6]:
# Data preprocessing with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
import os

# Set data directory
data_dir = r'D:\Food Image processing\food-101\images'

# List of all classes (food categories)
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# Count the number of images per class
class_counts = {cls: len(os.listdir(os.path.join(data_dir, cls))) for cls in classes}

# Sort classes by number of images
sorted_class_counts = dict(sorted(class_counts.items(), key=lambda item: item[1], reverse=True))

# Display the top N classes as text
top_n = 20
for idx, (cls, count) in enumerate(sorted_class_counts.items()):
    if idx < top_n:
        print(f"{idx + 1}. {cls}: {count} images")

In [ ]:
from PIL import Image

missing_files = []
corrupted_files = []

for cls in classes:
    class_dir = os.path.join(data_dir, cls)
    for filename in os.listdir(class_dir):
        file_path = os.path.join(class_dir, filename)
        try:
            # Open the image file
            img = Image.open(file_path)
            img.verify()  # Verify that it is a readable image
        except (IOError, SyntaxError) as e:
            print(f'Corrupted file: {file_path}')
            corrupted_files.append(file_path)
        except FileNotFoundError:
            print(f'Missing file: {file_path}')
            missing_files.append(file_path)

print(f"Total missing files: {len(missing_files)}")
print(f"Total corrupted files: {len(corrupted_files)}")

In [ ]:
import matplotlib.pyplot as plt
import random
from PIL import Image
import os

def show_random_images(data_dir, classes, num_classes=5, num_images=2):
    # Randomly select a subset of classes
    random_classes = random.sample(classes, min(num_classes, len(classes)))
    
    fig, axes = plt.subplots(num_images, len(random_classes), figsize=(15, num_images * 3))
    for i, cls in enumerate(random_classes):
        class_dir = os.path.join(data_dir, cls)
        image_files = os.listdir(class_dir)
        random_images = random.sample(image_files, min(num_images, len(image_files)))
        for j, image_file in enumerate(random_images):
            image_path = os.path.join(class_dir, image_file)
            img = Image.open(image_path)
            axes[j, i].imshow(img)
            axes[j, i].axis('off')
            if j == 0:
                axes[j, i].set_title(cls, fontsize=12)
    plt.tight_layout()
    plt.show()

show_random_images(data_dir, classes)

In [7]:
# Define the model using InceptionV3 as base
base_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(101, activation='softmax')  # Adjust the number of classes
])

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [8]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# Define callbacks
checkpoint_callback = ModelCheckpoint(
    'best_model.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,
    callbacks=[checkpoint_callback, early_stopping]
)